In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [2]:
agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    "../data/cal_1.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [5]:
res = agent.run("이번 주 토요일 일정 알려줘, 응답 결과는 json 형식으로 출력해줘")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nimport datetime\n\n# Filter rows for this week's Saturday\ntoday = datetime.date.today()\nsaturday = today + datetime.timedelta((5-today.weekday()) % 7)\nfiltered_df = df[df['calendar_start_date'].dt.date == saturday]\n\n# Convert dataframe to JSON\njson_result = filtered_df.to_json(orient='records')\njson_result"}`


[{"userId":1,"calendar_title":"3\uc8fc \ud68c\uc2dd","calendar_place":"\uc11c\uba74","calendar_start_date":1700265600000,"calendar_end_date":"2023-11-18"},{"userId":1,"calendar_title":"3\uc8fc \ud68c\uc2dd","calendar_place":"\uc11c\uba74","calendar_start_date":1700265600000,"calendar_end_date":"2023-11-18"}]이번 주 토요일 일정은 다음과 같습니다:

```json
[
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar_place": "서면",
    "calendar_start_date": 1700265600000,
    "calendar_end_date": "2023-11-18"
  },
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar

In [6]:
print(res)

이번 주 토요일 일정은 다음과 같습니다:

```json
[
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar_place": "서면",
    "calendar_start_date": 1700265600000,
    "calendar_end_date": "2023-11-18"
  },
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar_place": "서면",
    "calendar_start_date": 1700265600000,
    "calendar_end_date": "2023-11-18"
  }
]
```

위 결과는 JSON 형식으로 출력되었습니다.
